<a href="https://colab.research.google.com/github/centuryglass/glid-3-xl-expanded-inpainting/blob/colab-refactor/Inpainting_backend_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GLID-3-XL Inpainting UI backend server
Runs image generation operations for the inpainting UI, so that you can use the UI without a powerful GPU.

## Instructions:
1. Run all cells in this notebook, 
2. You will be asked to provide access to Google Drive. This is used to store your ngrok authentication token and optionally to load cached models. If you don't want to use drive, set `use_google_drive=False` below before running.
3. If you haven't run this before or you aren't using Google Drive, you will be prompted for an ngrok AuthToken in the first cell's output. This is required in order to connect to this server from the UI. You can sign up for an ngrok account for free at https://ngrok.com/, and you'll find your AuthToken at https://dashboard.ngrok.com/get-started/your-authtoken after logging in.
4. Wait for the server to start. This may take a few minutes, especially if you need to download model files.
5. Once you see `* Running on http://{Unique_session_id}.ngrok.io` in the output, the server is ready. Launch the inpainting UI, and enter in the ngrok.io link from the output when prompted for a server URL.

Unless you're using a paid ngrok account, connections are limited to 20/minute. Consider using smaller batch counts to avoid having to wait for this limit to reset.  If you are using a paid ngrok account, launch the UI with the `--fast_ngrok_connection` flag set to use fast image loading.


In [1]:
# Define settings, load models:

# Google Drive options:
# If use_google_drive=True, this will attempt to load your ngrok Authtoken and
# model files from Google Drive. If models aren't in drive they will be 
# downloaded normally.
use_google_drive=True
drive_model_dir="inpainting"

# If you're using google drive and haven't already uploaded models, set this
# to True and models will be uploaded to Drive so they can be loaded more
# quickly next time.  This will require something like 6-10 GB of available
# storage in Drive, depending on what models you're using.
save_missing_models_to_drive=False

# ML model options:
# model_path sets the primary model name. Unless you have a custom inpainting
# model that you're loading from Drive, you shouldn't change this.
model_path = 'inpaint.pt'

# Set clip_guidance to true for greater accuracy but reduced speed.
# This requires Colab Pro, free Colab instances don't have enough GPU memory.
# Currently untested, as I don't have Colab Pro.
clip_guidance=False
clip_guidance_scale=150

# Set clip model to use, valid options are RN50, RN101, RN50x4, RN50x16, 
# RN50x64, ViT-B/32, ViT-B/16, ViT-L/14. Most of these are not working when
# clip_guidance is true, but ViT-L/14 and RN50x16 might work on Colab Pro.
clip_model_name = 'ViT-L/14'

# Changing these will subtly alter the image generation process in ways I
# haven't bothered to identify yet. You should just be able to leave them as-is
# and still get good results:
cutn=16
ddim=False
ddpm=False # Not working currently, leave this as False for now



# Setup ngrok, google drive(if enabled):
import os
drive_path='/content/gdrive/MyDrive/'
drive_model_dir="inpainting"
if use_google_drive:
    from google.colab import drive
    drive.mount('/content/gdrive')
    if drive_model_dir:
      drive_path = drive_path + drive_model_dir + '/'
      if not os.path.exists(drive_path):
        os.mkdir(drive_path)

# load authtoken from Drive if possible, otherwise prompt for input:
authtoken = None
if use_google_drive and os.path.isfile(drive_path + 'ngrok-Authtoken.txt'):
  with open(drive_path + 'ngrok-Authtoken.txt', 'r', newline='') as tokenFile:
    authtoken = tokenFile.read()
if authtoken is None:
  from getpass import getpass
  authtoken = getpass('Enter ngrok Authtoken:')
  if use_google_drive:
    with open(drive_path + 'ngrok-Authtoken.txt', 'w', newline='') as tokenFile:
      tokenFile.write(authtoken)

# install dependencies:
import os, sys
!pip install ipywidgets omegaconf>=2.0.0 pytorch-lightning>=1.0.8 torch-fidelity einops ftfy regex tqdm transformers flask flask_cors flask_ngrok pyngrok
!pip install git+https://github.com/openai/CLIP.git
%cd /content
for compVisDep in ['taming-transformers', 'latent-diffusion']:
  if not os.path.exists(compVisDep):
    !git clone https://github.com/CompVis/{compVisDep}.git
    !pip install -e {compVisDep}
    sys.path.append(f'/content/{compVisDep}')
if not os.path.exists('glid-3-xl-expanded-inpainting'):
  !git clone https://github.com/centuryglass/glid-3-xl-expanded-inpainting
%cd glid-3-xl-expanded-inpainting
!git fetch origin
!git checkout origin/colab-refactor
!pip install -e .

# download required models:
if use_google_drive:
    from google.colab import drive
    drive.mount('/content/gdrive')

for model in ['bert.pt', 'kl-f8.pt', model_path]:
    if use_google_drive and not os.path.isfile(model) and os.path.isfile(f'{drive_path}{model}'):
      !cp '{drive_path}{model}' .
    if not os.path.isfile(model):
      !wget 'https://dall-3.com/models/glid-3-xl/{model}'
    if os.path.isfile(model):
      print(f"model file '{model}' obtained.")
      if use_google_drive and save_missing_models_to_drive and not os.path.isfile(f'{drive_path}{model}'):
        !cp '{model}' '{drive_path}{model}'
        if os.path.isfile(f'{drive_path}{model}'):
          print(f'saved {model} to google drive')
        else:
          print(f'error: unable to save {model} to google drive')
    else:
      print(f"error: failed to get required model file '{model}'!")

# load all models: 
import torch
import gc
gc.collect()
device = torch.device('cuda:0')
from startup.load_models import loadModels
model_params, model, diffusion, ldm, bert, clip_model, clip_preprocess, normalize = loadModels(
    device,
    model_path=model_path,
    clip_model_name=clip_model_name,
    clip_guidance=clip_guidance,
    ddpm=ddpm,
    ddim=ddim)

Mounted at /content/gdrive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-zqvtntc3
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-zqvtntc3
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369387 sha256=119bdded15767562e8f93cc587f0cca119dba81a8da09fcc2eb6200b3b63312b
  Stored in directory: /tmp/pip-ephem-wheel-cache-tz9_e6wa/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip
/content
Cloning into 'taming-transformers'...
remote: Enumerating objects: 1335, done.
remote: Total 1335 (delta 0), reused 0 (delta 0), pack-reused 1335
Receiving objects: 100% (1335/1335), 409.77 MiB | 37.59 MiB/s, done.
Resolving deltas: 100% (278/278), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/taming-transformers

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


loaded and configured primary model from inpaint.pt
loaded and configured latent diffusion model from kl-f8.pt


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

loaded and configured BERT model from bert.pt


100%|███████████████████████████████████████| 890M/890M [00:17<00:00, 52.4MiB/s]


loaded and configured CLIP model from ViT-L/14


In [ ]:
from flask_ngrok import run_with_ngrok
from colabFiles.server import startServer
app = startServer(device, model_params, model, diffusion, ldm, bert, clip_model, clip_preprocess, normalize)
run_with_ngrok(app)
app.run()


Starting server...
 * Serving Flask app "colabFiles.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e024-35-247-18-54.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Jul/2022 19:54:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2022 19:56:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2022 19:57:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2022 19:58:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2022 19:58:18] "POST / HTTP/1.1" 200 -


  0%|          | 0/27 [00:00<?, ?it/s]

...

127.0.0.1 - - [09/Jul/2022 20:09:32] "GET //sample HTTP/1.1" 200 -


Created 00014 at 1657397371.996573


127.0.0.1 - - [09/Jul/2022 20:09:33] "GET //sample/00010 HTTP/1.1" 200 -


Created 00010 at 1657397375.602419
Created 00011 at 1657397375.602419
Created 00012 at 1657397375.602419
Created 00013 at 1657397375.602419
Created 00014 at 1657397375.602419
Created 00010 at 1657397379.201859
Created 00011 at 1657397379.201859
Created 00012 at 1657397379.201859
Created 00013 at 1657397379.201859
Created 00014 at 1657397379.201859
Created 00010 at 1657397382.784045
Created 00011 at 1657397382.784045
Created 00012 at 1657397382.784045
Created 00013 at 1657397382.784045
Created 00014 at 1657397382.784045
Created 00010 at 1657397386.362168
Created 00011 at 1657397386.362168
Created 00012 at 1657397386.362168
Created 00013 at 1657397386.362168
Created 00014 at 1657397386.362168
Created 00010 at 1657397387.59228
Created 00011 at 1657397387.59228
Created 00012 at 1657397387.59228
Created 00013 at 1657397387.59228
Created 00014 at 1657397387.59228


127.0.0.1 - - [09/Jul/2022 20:10:22] "GET //sample HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2022 20:10:22] "GET //sample/00010 HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2022 20:10:22] "GET //sample/00011 HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2022 20:10:22] "GET //sample/00012 HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2022 20:10:23] "GET //sample/00013 HTTP/1.1" 200 -
127.0.0.1 - - [09/Jul/2022 20:10:23] "GET //sample/00014 HTTP/1.1" 200 -
